# Clustering Crypto

In [ ]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Deliverable 1: Preprocessing the Data for PCA

In [ ]:
# Load the crypto_data.csv dataset.
file_path = "resources/crypto_data.csv"
crypto_df = pd.read_csv(file_path)
crypto_df

In [ ]:
# Keep all the cryptocurrencies that are being traded.
crypto_df[crypto_df['IsTrading'] == True]

In [ ]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df[crypto_df['Algorithm'] != 'N/A']

In [ ]:
# Remove the "IsTrading" column. 
crypto_df.drop(columns=['IsTrading'], inplace=True)

In [ ]:
# Remove rows that have at least 1 null value.
crypto_df.dropna()

In [ ]:
# Keep the rows where coins are mined.
crypto_df.loc[crypto_df['TotalCoinsMined'] != 0]

In [ ]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_df[['CoinName']]

In [ ]:
crypto_df.set_index('Unnamed: 0')

In [ ]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns=['Unnamed: 0', 'CoinName'], inplace=True)

In [ ]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(data=crypto_df, columns=['Unnamed: 0', 'Algorithm', 'ProofType'])

In [ ]:
# Standardize the data with StandardScaler().
scaled_crypto_df = StandardScaler().fit_transform(X)

### Deliverable 2: Reducing Data Dimensions Using PCA

In [ ]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_df_pca = pca.fit_transform(X)

In [ ]:
# Create a DataFrame with the three principal components.
crypto_df_pca = pd.DataFrame(
data=crypto_df_pca, columns=['PC 1', 'PC 2', 'PC 3']
)
crypto_df_pca.head()

### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [ ]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_df_pca)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

Running K-Means with `k=4`

In [ ]:
# Initialize the K-Means model.
model = KMeans(n_clusters=3, random_state=0)

# Fit the model
model.fit(crypto_df_pca)

# Predict clusters
predictions = model.predict(crypto_df_pca)


In [ ]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.DataFrame({
    "Algorithm": crypto_df.Algorithm,
    "ProofType": crypto_df.ProofType,
    "TotalCoinsMined": crypto_df.TotalCoinsMined,
    "TotalCoinSupply": crypto_df.TotalCoinSupply,
    "PC 1": crypto_df_pca['PC 1'],
    "PC 2": crypto_df_pca['PC 2'],
    "PC 3": crypto_df_pca['PC 3'],
    },
)
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = crypto_df['CoinNames']

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = predictions

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [ ]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    five_clusters,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="class",
    symbol="class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [ ]:
# Create a table with tradable cryptocurrencies.
columns = ['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class']
clustered_df.hvplot.table(columns)

In [ ]:
# Print the total number of tradable cryptocurrencies.
scaled_clustered_df.count()

In [ ]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaled_clustered_df = StandardScaler().fit_transform(clustered_df)

In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
new_clustered_df['index'] = scaled_clustered_df['index']

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
new_clustered_df['CoinName'] = scaled_clustered_df['CoinNames']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
new_clustered_df['Class'] = scaled_clustered_df['Class']

new_clustered_df.head(10)

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
new_clustered_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="class")